In [0]:
# Importing Libraries

In [0]:
%tensorflow_version 1.x

In [0]:
import pandas as pd
import numpy as np

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
dir_path="/content/drive/My Drive/Colab Notebooks/AppliedAI/HumanActivityRecognition/HAR/"

In [0]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [0]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [0]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [0]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'{dir_path}UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [10]:
print(dir_path)

/content/drive/My Drive/Colab Notebooks/AppliedAI/HumanActivityRecognition/HAR/


In [0]:
def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'{dir_path}UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    # return pd.get_dummies(y).as_matrix()
    return pd.get_dummies(y).values

In [0]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [13]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
from keras import backend as K
tf.set_random_seed(42)

Using TensorFlow backend.


In [0]:
# Configuring a session
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [0]:
# Import Keras
from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [0]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM, Conv1D, MaxPooling1D,InputLayer,TimeDistributed,\
                          ConvLSTM2D
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint

In [0]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [0]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [40]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()


In [41]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(7352, 128, 9)
(2947, 128, 9)
(7352, 6)
(2947, 6)


In [42]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


In [0]:
#container to hold all the parameters and results
d = {
        "layers"    : [],
        "n_hidden"  : [],
        "dropout"  : [],
        "parameters_trained" : [],
        "train_acc" : [],
        "test_acc" : []
    }

In [0]:
def get_trainable_parameters(model):
    trainable_count = int(np.sum([K.count_params(p) for p in set(model.trainable_weights)]))
    return trainable_count

### Defining the Architecture of LSTM

In [0]:
def define_compile_model(n_hidden=[32],layer=1,dropout=[0.5]):
    assert(len(n_hidden)==layer)
    global timesteps,input_dim,n_classes,d
    ret_seq = True if layer > 1 else False
    d["layers"].append(layer)
    d["n_hidden"].append(n_hidden)
    d["dropout"].append(dropout)
    model = Sequential()
    model.add(LSTM(n_hidden[0],input_shape=(timesteps, input_dim),
                   return_sequences=ret_seq))
    model.add(BatchNormalization())
    model.add(Dropout(dropout[0]))
    for i in range(1,layer):
        if i==layer-1:
            ret_seq = False
        model.add(LSTM(n_hidden[i],return_sequences=ret_seq))
        model.add(BatchNormalization())
        model.add(Dropout(dropout[i]))
    model.add(Dense(n_classes,activation='sigmoid'))
    model.summary()
    # Compiling the model
    model.compile(loss='categorical_crossentropy',optimizer='rmsprop',
                  metrics=['accuracy'])    # also tried adam
    return model

In [25]:
# https://machinelearningmastery.com/how-to-develop-rnn-models-for-human-activity-recognition-time-series-classification/
# Trying out different custom models

# Model 1 : LSTM 

model = Sequential()
model.add(LSTM(128, input_shape=(timesteps,input_dim),return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(128))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(n_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics=['accuracy'])




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [0]:
# Model 2: CNN-LSTM

n_steps, n_length = 4, 32
X_train = X_train.reshape((X_train.shape[0], n_steps, n_length, input_dim))
X_test = X_test.reshape((X_test.shape[0], n_steps, n_length, input_dim))

In [0]:
model = Sequential()
model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, 
                  activation='relu'), input_shape=(None,n_length,input_dim)))
model.add(TimeDistributed(Conv1D(filters=32, kernel_size=3,
                  activation='relu')))
model.add(TimeDistributed(Dropout(0.5)))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(100))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(n_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics=['accuracy'])

In [0]:
# Model 3: ConvLSTM2D

timesteps, input_dim, n_classes = X_train.shape[1], X_train.shape[2],\
                                  Y_train.shape[1]
# reshape into subsequences (samples, time steps, rows, cols, channels)
n_steps, n_length = 4, 32
X_train = X_train.reshape((X_train.shape[0], n_steps, 1, n_length, input_dim))
X_test = X_test.reshape((X_test.shape[0], n_steps, 1, n_length, input_dim))

In [0]:
# define model
model = Sequential()
model.add(ConvLSTM2D(filters=64, kernel_size=(1,3), 
                     activation='relu', 
                     input_shape=(n_steps, 1, n_length, input_dim)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(n_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics=['accuracy'])

In [46]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_11 (TimeDis (None, None, 30, 64)      1792      
_________________________________________________________________
time_distributed_12 (TimeDis (None, None, 28, 32)      6176      
_________________________________________________________________
time_distributed_13 (TimeDis (None, None, 28, 32)      0         
_________________________________________________________________
time_distributed_14 (TimeDis (None, None, 14, 32)      0         
_________________________________________________________________
time_distributed_15 (TimeDis (None, None, 448)         0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               219600    
_________________________________________________________________
dropout_9 (Dropout)          (None, 100)              

### fit  and predict model .. 

In [0]:
def fit_model(X_train,y_train,batch_size,X_test,y_test,epochs,model):
    global d
    # Training the modela
    model.fit(X_train,Y_train,batch_size=batch_size,
              validation_data=(X_test, Y_test),epochs=epochs)
    d["parameters_trained"].append(get_trainable_parameters(model))
    d["train_acc"].append(model.history.history['acc'][-1])
    # Confusion Matrix
    print(confusion_matrix(Y_test, model.predict(X_test)))
    return model

def predict_model(X_test,y_test):
    #evaluate the model :: socre[1] contains the accuracy for the given data ...
    score = model.evaluate(X_test,Y_test)
    d["test_acc"].append(score[1])
    return model

### LSTM with layer = 1 , n_hidden = 32 and dropout = [0.5]

In [0]:
model = define_compile_model(n_hidden=[32,32,32],layer=3,
                             dropout=[0.2,0.2,0.2])




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128, 32)           5376      
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 32)           128       
_________________________________________________________________
dropout_1 (Dropout)          (None, 128, 32)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128, 32)           8320      
_________________________________________________________________
batch_normalization_2 (Batch (None, 128, 32)           128       
_________________________________________________________________
dropout_2 (Dropout)          (None, 128, 32)           0         
__

In [47]:
model = fit_model(X_train,Y_train,batch_size,X_test,Y_test,15,model)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 7352 samples, validate on 2947 samples
Epoch 1/15




7352/7352 [==============================] - 18s 2ms/step - loss: 0.4835 - acc: 0.8032 - val_loss: 0.5491 - val_acc: 0.8310
Epoch 2/15
7352/7352 [==============================] - 5s 626us/step - loss: 0.1771 - acc: 0.9338 - val_loss: 0.4791 - val_acc: 0.8500
Epoch 3/15
7352/7352 [==============================] - 5s 629us/step - loss: 0.1387 - acc: 0.9441 - val_loss: 0.3689 - val_acc: 0.8945
Epoch 4/15
7352/7352 [==============================] - 5s 643us/step - loss: 0.1294 - acc: 0.9472 - val_loss: 0.6042 - val_acc: 0.8551
Epoch 5/15
7352/7352 [==============================] - 5s 638us/step - loss: 0.1340 - acc: 0.9506 - val_loss: 0.4288 - val_acc: 0.8948
Epoch 6/15
7352/7352 [==============================] - 5s 633us/step - loss: 0.1163 - acc: 0.9508 - val_loss: 0.4495 - val_acc: 0.8877
Epoch 7/15
7352/7352 [===============

In [49]:
model = predict_model(X_test,Y_test)

2947/2947 [==============================] - 0s 87us/step


### printing d value 

In [0]:
print(d)

{'layers': [], 'n_hidden': [], 'dropout': [], 'parameters_trained': [302982, 302982], 'train_acc': [0.9606909684439608, 0.9639553862894451], 'test_acc': [0.9104173736002714, 0.8730912792670512]}


In [0]:
print(d)

{'layers': [], 'n_hidden': [], 'dropout': [], 'parameters_trained': [249026, 238274, 238274], 'train_acc': [0.9560663764961915, 0.9575625680087051, 0.9672198041349293], 'test_acc': [0.9121140142517815, 0.9243298269426535, 0.8598574821852731]}


- With a simple 2 layer architecture we got 90.09% accuracy and a loss of 0.30
- We can further imporve the performace with Hyperparameter tuning

In [0]:
##################################
# hyper paratemer tunning
# hidden units , dropout, lstm layers (but be careful that it will not overfit) (use different dropout rate .. )
# use pretty table to conclude epochs can also be changed ..
# pretty table columns - layers  , n_hidden, dropout used for each layer, train_Acc, test,Acc
####################################

from prettytable import PrettyTable
x = PrettyTable()
x.field_names = ["layers", "n_hidden", "dropout", "parameters_trained","train_acc","test_acc"]
for i in range(len(d["layers"])):
    x.add_row([d["layers"][i],d["n_hidden"][i],d["dropout"][i],d["parameters_trained"][i],d["train_acc"][i],d["test_acc"][i]])
print(x)

+--------+----------+-----------------+--------------------+--------------------+--------------------+
| layers | n_hidden |     dropout     | parameters_trained |     train_acc      |      test_acc      |
+--------+----------+-----------------+--------------------+--------------------+--------------------+
|   1    |    32    |      [0.5]      |        5574        | 0.9434167573449401 | 0.8842891075670173 |
|   2    |    32    |    [0.5, 0.5]   |       13894        | 0.9473612622415669 | 0.8934509670851714 |
|   3    |    64    | [0.3, 0.3, 0.3] |       85382        | 0.9533460282916213 | 0.9131319986426875 |
+--------+----------+-----------------+--------------------+--------------------+--------------------+


In [0]:
from prettytable import PrettyTable
x = PrettyTable()
x.field_names = ["layers", "n_hidden", "dropout", "parameters_trained","train_acc","test_acc"]
for i in range(len(d["layers"])):
    x.add_row([d["layers"][i],d["n_hidden"][i],d["dropout"][i],d["parameters_trained"][i],d["train_acc"][i],d["test_acc"][i]])
print(x)

+--------+------------------+----------------------+--------------------+--------------------+--------------------+
| layers |     n_hidden     |       dropout        | parameters_trained |     train_acc      |      test_acc      |
+--------+------------------+----------------------+--------------------+--------------------+--------------------+
|   1    |       [32]       |        [0.5]         |        5574        | 0.9434167573449401 | 0.8842891075670173 |
|   1    |       [64]       |        [0.5]         |       19462        | 0.9480413492927094 | 0.8842891075670173 |
|   1    |       [64]       |        [0.3]         |       19462        | 0.9500816104461371 | 0.9267051238547676 |
|   1    |      [128]       |        [0.5]         |       71686        | 0.6228237214363439 | 0.168306752629793  |
|   1    |      [128]       |        [0.5]         |       71686        | 0.9465451577801959 | 0.9280624363759755 |
|   2    |     [32, 32]     |      [0.4, 0.3]      |       14022        

In [0]:
from prettytable import PrettyTable
x = PrettyTable()
x.field_names = ["layers", "n_hidden", "dropout", "parameters_trained","train_acc","test_acc"]
for i in range(len(d["layers"])):
    x.add_row([d["layers"][i],d["n_hidden"][i],d["dropout"][i],d["parameters_trained"][i],d["train_acc"][i],d["test_acc"][i]])
print(x)

+--------+--------------+-----------------+--------------------+--------------------+--------------------+
| layers |   n_hidden   |     dropout     | parameters_trained |     train_acc      |      test_acc      |
+--------+--------------+-----------------+--------------------+--------------------+--------------------+
|   3    | [32, 32, 32] | [0.2, 0.2, 0.2] |       22406        | 0.9413764961915125 | 0.8988802171700034 |
+--------+--------------+-----------------+--------------------+--------------------+--------------------+


In [0]:
table = PrettyTable(['layers','n_hidden','dropout','parameters_trained',
                     'train_acc','test_acc'])

In [0]:
table.add_row([4,[100,100,6],[0.5,0.5],54706,0.9163492927094669,0.9066847641669494])
table.add_row([3,[64,100,6],[0.5],249026,0.9560663764961915,0.9121140142517815])
table.add_row([7,[64,32,100,100,6],[0.5,0.5],238274,0.9572198041349293,0.9298574821852731])
table.add_row([7,[64,32,100,100,6],[0.5,0.5],238274,0.9675625680087051,0.9443298269426535])

In [0]:
print(table)

+--------+-----------------------+------------+--------------------+--------------------+--------------------+
| layers |        n_hidden       |  dropout   | parameters_trained |     train_acc      |      test_acc      |
+--------+-----------------------+------------+--------------------+--------------------+--------------------+
|   4    |     [100, 100, 6]     | [0.5, 0.5] |       54706        | 0.9163492927094669 | 0.9066847641669494 |
|   3    |      [64, 100, 6]     |   [0.5]    |       249026       | 0.9560663764961915 | 0.9121140142517815 |
|   7    | [64, 32, 100, 100, 6] | [0.5, 0.5] |       238274       | 0.9572198041349294 | 0.9298574821852731 |
|   7    | [64, 32, 100, 100, 6] | [0.5, 0.5] |       238274       | 0.9675625680087051 | 0.9443298269426535 |
+--------+-----------------------+------------+--------------------+--------------------+--------------------+


In [0]:
# Conclusions

# We clearly see that the model performs fairly well on shallow LSTM layers
# but as we increase the depth of the model architecture it suffers overfitting

# Also, we experimented by adding large values of dropouts, and reducing the
# number of training epochs but still the deep networks are not gettings any
# better substantially

# The best model having two LSTM layers each having 32 hidden units

In [0]:
# At last we experimented with 2 other kinds of custome models instead of 
# vanilla LSTMS's. They were CNN-LSTM and ConvLSTM. We were able to achieve our
# target accuracy of >94% using the CNN-LSTM model after palying around with 
# the hyperparams and number of epochs to avoid overfitting 

In [0]:
# Trying out a completely new Github implementation

In [0]:
# Using code directly from :
# https://github.com/heeryoncho/sensors2018cnnhar/

In [0]:
from scipy import ndimage

In [0]:
dir_path = dir_path+'UCI_HAR_Dataset/'

In [11]:
print(dir_path)

/content/drive/My Drive/Colab Notebooks/AppliedAI/HumanActivityRecognition/HAR/UCI_HAR_Dataset/


In [0]:
def load_x(train_or_test):
    global dir_path
    if train_or_test is "train":
        x_path = dir_path + 'train/X_train.txt'
    elif train_or_test is "test":
        x_path = dir_path + 'test/X_test.txt'

    with open(x_path) as f:
        container = f.readlines()

    result = []
    for line in container:
        tmp1 = line.strip()
        tmp2 = tmp1.replace('  ', ' ')     # removes inconsistent blank spaces
        tmp_ary = list(map(float, tmp2.split(' ')))
        result.append(tmp_ary)
    return np.array(result)

In [0]:
def load_y(train_or_test):
    global dir_path
    if train_or_test is "train":
        y_path = dir_path + 'train/y_train.txt'
    elif train_or_test is "test":
        y_path = dir_path + 'test/y_test.txt'

    with open(y_path) as f:
        container = f.readlines()

    result = []
    for line in container:
        num_str = line.strip()
        result.append(int(num_str))
    return np.array(result)

In [0]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
import numpy as np
import random
from numpy.random import seed
from sklearn.metrics import accuracy_score,confusion_matrix
from keras.optimizers import Adam
from keras.models import Sequential, load_model
from keras.layers import Conv1D, MaxPooling1D, Dense, Flatten, Dropout
from keras.callbacks import ModelCheckpoint
from keras.utils import plot_model
import keras.backend as K

In [0]:
X_train_all = load_x("train")
y_train_all = load_y("train")

X_test_all = load_x("test")
y_test_all = load_y("test")

In [31]:
# --------------------------------------
# Only dynamic HAR data are selected
# --------------------------------------

# Select dynamic HAR train data

dynamic_1 = np.where(y_train_all == 1)[0]
dynamic_2 = np.where(y_train_all == 2)[0]
dynamic_3 = np.where(y_train_all == 3)[0]
dynamic = np.concatenate([dynamic_1, dynamic_2, dynamic_3])

X_train = X_train_all[dynamic]
y_train = y_train_all[dynamic]

# Convert (1, 2, 3) labels to (0, 1, 2)
y_train  = y_train - 1

print("\n+++ DATA STATISTICS +++\n")
print("train_dynamic shape: ", X_train.shape)

# Select dynamic HAR test data

dynamic_1 = np.where(y_test_all == 1)[0]
dynamic_2 = np.where(y_test_all == 2)[0]
dynamic_3 = np.where(y_test_all == 3)[0]
dynamic = np.concatenate([dynamic_1, dynamic_2, dynamic_3])

X_test = X_test_all[dynamic]
y_test = y_test_all[dynamic]

# Convert (1, 2, 3) labels to (0, 1, 2)
y_test  = y_test - 1

print("test_dynamic shape: ", X_test.shape)


+++ DATA STATISTICS +++

train_dynamic shape:  (3285, 561)
test_dynamic shape:  (1387, 561)


In [32]:
dir_path

'/content/drive/My Drive/Colab Notebooks/AppliedAI/HumanActivityRecognition/HAR/UCI_HAR_Dataset/'

In [35]:
# Display dynamic model accuracy

print("\n+++ DYNAMIC MODEL ACCURACY (See Table 8 in paper) +++\n")

model_path = dir_path+"models/dynamic.hdf5"
model = load_model(model_path)

pred_train = model.predict(np.expand_dims(X_train, axis=2), batch_size=32)
print("------ TRAIN ACCURACY ------")
print(accuracy_score(y_train, np.argmax(pred_train, axis=1)))
print(confusion_matrix(y_train, np.argmax(pred_train, axis=1)))

pred_test = model.predict(np.expand_dims(X_test, axis=2), batch_size=32)
print("------ TEST ACCURACY ------")
print(accuracy_score(y_test, np.argmax(pred_test, axis=1)))
print(confusion_matrix(y_test, np.argmax(pred_test, axis=1)))



+++ DYNAMIC MODEL ACCURACY (See Table 8 in paper) +++






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.











/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:350: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


------ TRAIN ACCURACY ------
0.9863013698630136
[[1223    3    0]
 [   4 1038   31]
 [   0    7  979]]
------ TEST ACCURACY ------
0.9798125450612833
[[495   0   1]
 [  2 469   0]
 [  2  23 395]]


In [38]:
static_1 = np.where(y_train_all == 4)[0]
static_2 = np.where(y_train_all == 5)[0]
static_3 = np.where(y_train_all == 6)[0]
static = np.concatenate([static_1, static_2, static_3])

X_train = X_train_all[static]
y_train = y_train_all[static]

# Convert (4, 5, 6) labels to (0, 1, 2)
y_train  = y_train - 4

print("\n+++ DATA STATISTICS +++\n")
print("train_static shape: ", X_train.shape)

# Select static HAR test data

static_1 = np.where(y_test_all == 4)[0]
static_2 = np.where(y_test_all == 5)[0]
static_3 = np.where(y_test_all == 6)[0]
static = np.concatenate([static_1, static_2, static_3])

X_test = X_test_all[static]
y_test = y_test_all[static]

# Convert (4, 5, 6) labels to (0, 1, 2)
y_test  = y_test - 4

print("test_static shape: ", X_test.shape)


# Display static model accuracy

print("\n+++ STATIC MODEL ACCURACY (See Table 8 in paper) +++\n")

model_path = dir_path+"models/static.hdf5"
model = load_model(model_path)

pred_train = model.predict(np.expand_dims(X_train, axis=2), batch_size=32)
print("------ TRAIN ACCURACY ------")
print(accuracy_score(y_train, np.argmax(pred_train, axis=1)))
print(confusion_matrix(y_train, np.argmax(pred_train, axis=1)))

pred_test = model.predict(np.expand_dims(X_test, axis=2), batch_size=32)
print("------ TEST ACCURACY ------")
print(accuracy_score(y_test, np.argmax(pred_test, axis=1)))
print(confusion_matrix(y_test, np.argmax(pred_test, axis=1)))



+++ DATA STATISTICS +++

train_static shape:  (4067, 561)
test_static shape:  (1560, 561)

+++ STATIC MODEL ACCURACY (See Table 8 in paper) +++



/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:350: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


------ TRAIN ACCURACY ------
0.9923776739611507
[[1275   11    0]
 [  20 1354    0]
 [   0    0 1407]]
------ TEST ACCURACY ------
0.966025641025641
[[453  38   0]
 [ 14 518   0]
 [  1   0 536]]
